# Imports

In [ ]:
import sys
import os

In [ ]:
# install in your environment
!{sys.executable} -m pip install langchain openai
!{sys.executable} -m pip install langchain_community openai
!{sys.executable} -m pip install --upgrade langchain langchain_community openai
!{sys.executable} -m pip install --upgrade langchain langchain-openai openai
!{sys.executable} -m pip install -U langchain-ollama

from langchain_community.llms import OpenAI
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate

from langchain_openai import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_ollama import ChatOllama

In [ ]:
from dsi.dsi import DSI

## AI Function

In [ ]:
# a function to ask chatgpt for queries
def ask_chatgpt_for_sql(schema, user_request, open_ai_key, model_name="gpt-5"):
    # Set key
    os.environ["OPENAI_API_KEY"] = open_ai_key
    
    # Use LLM to extract queries
    prompt_template = (
        "You are an expert SQL assistant.\n"
        "Here is the schema:\n{schema}\n\n"
        "Generate a SQL query for the following request:\n{user_request}\n\n"
        "Only output the SQL query on one line please."
    )
    
    prompt = PromptTemplate(
        input_variables=["schema", "user_request"],
        template=prompt_template
    )
    
    # Initialize the LLM
    llm = ChatOpenAI(model_name=model_name, temperature=0)
    
    # Create the chain
    chain = prompt | llm | StrOutputParser()
    response = chain.invoke({
        "schema": schema, 
        "user_request": user_request
    })

    print(response)
    return response.strip()

In [ ]:
def ask_ollama_for_sql(schema, user_request, model_name="mistral:latest"):
    # No API key needed for Ollama (it runs locally)

    prompt_template = (
        "You are an expert SQL assistant.\n"
        "Here is the schema:\n{schema}\n\n"
        "Generate a SQL query for the following request:\n{user_request}\n\n"
        "Only output the SQL query on one line please."
    )

    prompt = PromptTemplate(
        input_variables=["schema", "user_request"],
        template=prompt_template
    )

    # Initialize the local chat model served by Ollama
    llm = ChatOllama(model=model_name, temperature=0)

    # LCEL chain
    chain = prompt | llm | StrOutputParser()

    # NOTE: your original snippet had {user_query}; use user_request instead.
    response = chain.invoke({
        "schema": schema,
        "user_request": user_request
    })

    print(response)
    return response.strip()

# Load the data

In [ ]:
store = DSI(backend_name = "Sqlite")

In [ ]:
store.read("wildfire/wildfiredata.csv", 'CSV', table_name='wildfiredata')
store.summary()

In [ ]:
# Get the schema
store.query("SELECT name, type, sql FROM sqlite_master WHERE sql NOT NULL ORDER BY type, name")

# Use AI to extract data

In [ ]:
schema = "CREATE TABLE wildfiredata (wind_speed INTEGER, wdir INTEGER, smois FLOAT, fuels VARCHAR, ignition VARCHAR, safe_unsafe_ignition_pattern VARCHAR, safe_unsafe_fire_behavior VARCHAR, does_fire_meet_objectives VARCHAR, rationale_if_unsafe VARCHAR, burned_area INTEGER, FILE VARCHAR)"

## Query AI for data

In [ ]:
user_question = "Find all entries where wind speed is above 10"
sql_query_1 = ask_chatgpt_for_sql(schema, user_question, os.getenv("OPENAI_API_KEY"),  model_name="gpt-5")

In [ ]:
store.query(sql_query_1)

In [ ]:
user_question = "Find all entries where wind speed is above 10 and show it in ascending order of burned area size, and show only the top 5"
sql_query_2 = ask_ollama_for_sql(schema, user_question)

In [ ]:
store.query("SELECT * FROM wildfiredata WHERE wind_speed > 10 ORDER BY burned_area ASC LIMIT 5;")